# Problem Statement
- Build a gesture recognition system using deep learning models such as CNNs and RNNs.
- Apply the system in the context of human-computer interaction or gaming.

Link to data: https://www.kaggle.com/datasets/gti-upm/leapgestrecog/data

## Import Required Libraries and Load Dataset 

In [36]:
import os
import cv2
import numpy as np
import kagglehub
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image

# Download latest version
path = kagglehub.dataset_download("gti-upm/leapgestrecog")

print("Path to dataset files:", path)

Path to dataset files: /home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1


## Preprocess and Visualize Data

In [ ]:
# Parameters
IMG_SIZE = 64  # Resize images to 64x64 pixels
num_classes = 10  # Number of gesture classes
data = []
labels = []

# Update the path to the correct directory
path = '/home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapgestrecog/leapGestRecog'

# Check contents again to confirm
print("Updated Root Directory Contents:", os.listdir(path))

for subject in os.listdir(path):
    subject_path = os.path.join(path, subject)
    
    if os.path.isdir(subject_path):
        print(f"Found subject directory: {subject_path}")
        for gesture in os.listdir(subject_path):
            gesture_path = os.path.join(subject_path, gesture)
            
            if os.path.isdir(gesture_path):  # Check if it's a valid gesture folder
                for img_name in os.listdir(gesture_path):
                    img_path = os.path.join(gesture_path, img_name)
                    
                    # Load image in grayscale
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
                    # Resize the image
                    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    
                    # Append image and label to the lists
                    data.append(img_resized)
                    # Assuming the gesture folder name follows the pattern '01_palm', '02_l', etc.
                    labels.append(int(gesture.split('_')[0]))  # Use the gesture number as label
                    
# Change this so it will display one of each of the gestures instead of all the same
"""num_images_to_display = 5
for i in range(min(num_images_to_display, len(data))):
    plt.imshow(cv2.cvtColor(data[i], cv2.COLOR_BGR2RGB))
    plt.title(f"Gesture {i}")
    plt.axis('off')  # Hide axes
    plt.show()
"""

# Convert data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Normalize data (scale pixel values between 0 and 1)
data = data.astype("float32") / 255.0

# Reshape data to add channel dimension (grayscale images have 1 channel)
data = np.expand_dims(data, axis=-1)
print(np.unique(labels))

# One-hot encode labels
labels = tf.keras.utils.to_categorical(labels-1, num_classes)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Updated Root Directory Contents: ['04', '06', '02', '03', '07', '08', '05', '09', '00', '01']
Found subject directory: /home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapgestrecog/leapGestRecog/04
Found subject directory: /home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapgestrecog/leapGestRecog/06
Found subject directory: /home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapgestrecog/leapGestRecog/02
Found subject directory: /home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapgestrecog/leapGestRecog/03
Found subject directory: /home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapgestrecog/leapGestRecog/07
Found subject directory: /home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapgestrecog/leapGestRecog/08
Found subject directory: /home/codespace/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapgestrecog/leapGe

## Create Model

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

model = Sequential()

# Add a convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D((2, 2)))

# Add another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output from the previous layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization

# Output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-17 18:40:24.920234: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,625,866 (6.20 MB)

 Trainable params: 1,625,866 (6.20 MB)

...

  2/500 ━━━━━━━━━━━━━━━━━━━━ 45s 91ms/step - accuracy: 0.0391 - loss: 2.2965

2025-02-17 18:40:27.077768: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.
2025-02-17 18:40:27.185438: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.
2025-02-17 18:40:27.276712: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.


  5/500 ━━━━━━━━━━━━━━━━━━━━ 54s 110ms/step - accuracy: 0.0765 - loss: 2.3001

2025-02-17 18:40:27.394585: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.


500/500 ━━━━━━━━━━━━━━━━━━━━ 48s 93ms/step - accuracy: 0.6607 - loss: 1.0015 - val_accuracy: 0.9977 - val_loss: 0.0119
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 48s 96ms/step - accuracy: 0.9754 - loss: 0.0761 - val_accuracy: 0.9990 - val_loss: 0.0034
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 48s 96ms/step - accuracy: 0.9885 - loss: 0.0355 - val_accuracy: 0.9992 - val_loss: 0.0017
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 95ms/step - accuracy: 0.9907 - loss: 0.0299 - val_accuracy: 0.9995 - val_loss: 0.0016
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 95ms/step - accuracy: 0.9925 - loss: 0.0223 - val_accuracy: 0.9995 - val_loss: 0.0039
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 95ms/step - accuracy: 0.9947 - loss: 0.0163 - val_accuracy: 0.9998 - val_loss: 0.0013
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 95ms/step - accuracy: 0.9934 - loss: 0.0198 - val_accuracy: 0.9995 - val_loss: 0.0020
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 80s 91ms/step - accuracy: 0.9963 - loss: 0.0137 - val_accurac

## Evaluate and Visualize Predictions

In [45]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the test accuracy
print(f"Test accuracy: {test_acc:.4f}")

import matplotlib.pyplot as plt

# Make predictions on the test set
predictions = model.predict(X_test)

# Select a random sample of test images to visualize
num_samples = 5
indices = np.random.choice(range(len(X_test)), size=num_samples)

# Visualize the predictions
plt.figure(figsize=(10, 10))
for i, idx in enumerate(indices):
    plt.subplot(1, num_samples, i+1)
    
    # Display the image
    plt.imshow(X_test[idx].reshape(64, 64), cmap='gray')  # Reshape to 64x64 for visualization
    plt.axis('off')  # Hide axes
    
    # Get the predicted label (highest probability)
    predicted_label = np.argmax(predictions[idx])
    
    # Get the true label
    true_label = np.argmax(y_test[idx])
    
    # Display the true and predicted labels
    plt.title(f"True: {true_label}, Pred: {predicted_label}")
    
plt.show()



125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9998 - loss: 0.0017
Test accuracy: 0.9995
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step


ImportError: cannot import name 'backend_agg' from 'matplotlib.backends' (/home/codespace/.local/lib/python3.12/site-packages/matplotlib/backends/__init__.py)

## Live Demo